In [ ]:
import json

def create_training_example(papers, paper_id):
    """
    Given a list of paper records (as dictionaries) and a specific paper_id,
    returns a dictionary with the keys:
        {
          "messages": [
            { "role": "system", "content": "...system content..." },
            { "role": "user", "content": "...user content..." },
            { "role": "assistant", "content": "...assistant content..." }
          ]
        }
    which can be used as a single training example for instruction fine-tuning.
    """
    # 1) Find the matching paper by paper_id
    paper_data = next((p for p in papers if p["paper_id"] == paper_id), None)
    if not paper_data:
        raise ValueError(f"No paper found with paper_id={paper_id}")

    # 2) Build the system message
    system_message = (
        "You are a helpful assistant that extracts a structured summary from a paper. "
        "Please return only valid JSON with the following fields:\n"
        "Soundness, Presentation, Contribution, Rating, Confidence (all integers)\n"
        "Strengths, Weaknesses, and Questions (all strings).\n"
        "No additional keys. No extra text."
    )

    # 3) Build the user message
    #    - Insert the paper summary
    #    - Provide definitions for each integer category
    user_message = (
        f"Please read the following paper summary and produce reviewer scores.\n\n"
        f"--- PAPER SUMMARY ---\n\n{paper_data['summary']}\n\n"
        f"--- REQUIRED CATEGORIES ---\n"
        f"We need the following fields in the output:\n"
        f"1) Soundness\n"
        f"2) Presentation\n"
        f"3) Contribution\n"
        f"4) Rating\n"
        f"5) Confidence\n"
        f"6) Strengths\n"
        f"7) Weaknesses\n"
        f"8) Questions\n\n"
        f"--- POSSIBLE VALUES & DEFINITIONS ---\n\n"
        f"Soundness, Presentation, and Contribution (1–4):\n"
        f"- 1: poor\n"
        f"- 2: fair\n"
        f"- 3: good\n"
        f"- 4: excellent\n\n"
        f"Rating (1–10):\n"
        f"- 1: Trivial or wrong\n"
        f"- 2: Strong rejection\n"
        f"- 3: Clear rejection\n"
        f"- 4: Ok but not good enough - rejection\n"
        f"- 5: Marginally below acceptance threshold\n"
        f"- 6: Marginally above acceptance threshold\n"
        f"- 7: Good paper, accept\n"
        f"- 8: Top 50% of accepted papers, clear accept\n"
        f"- 9: Top 15% of accepted papers, strong accept\n"
        f"- 10: Top 5% of accepted papers, seminal paper\n\n"
        f"Confidence (1–5):\n"
        f"- 1: The reviewer's evaluation is an educated guess\n"
        f"- 2: The reviewer is willing to defend the evaluation, but might be wrong\n"
        f"- 3: The reviewer is fairly confident that the evaluation is correct\n"
        f"- 4: The reviewer is confident but not absolutely certain that the evaluation is correct\n"
        f"- 5: The reviewer is absolutely certain that the evaluation is correct and very familiar with the topic\n\n"
        f"--- OUTPUT FORMAT ---\n"
        f"Please produce only a JSON object with this exact structure (no extra keys):\n"
        f"{{\n"
        f"  \"Soundness\": <int>,\n"
        f"  \"Presentation\": <int>,\n"
        f"  \"Contribution\": <int>,\n"
        f"  \"Rating\": <int>,\n"
        f"  \"Confidence\": <int>,\n"
        f"  \"Strengths\": \"<string>\",\n"
        f"  \"Weaknesses\": \"<string>\",\n"
        f"  \"Questions\": \"<string>\"\n"
        f"}}\n\n"
        f"No explanations or markdown formatting. Only valid JSON."
    )

    # 4) Build the assistant message
    #    - The "response" in the paper data already looks like valid JSON,
    #      so we just serialize it. The assistant's content should be a JSON string.
    assistant_content = json.dumps(paper_data["response"], ensure_ascii=False)

    # 5) Construct the final training example
    training_example = {
        "messages": [
            {
                "role": "system",
                "content": system_message
            },
            {
                "role": "user",
                "content": user_message
            },
            {
                "role": "assistant",
                "content": assistant_content
            }
        ]
    }

    return training_example
